## Import PyTorch

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

## Initialize Hyper-parameters

In [2]:
input_size = 784       # The image size = 28 x 28 = 784
hidden_size = 500      # The number of nodes at the hidden layer
num_classes = 10       # The number of output classes. In this case, from 0 to 9
num_epochs = 5         # The number of times entire dataset is trained
batch_size = 100       # The size of input data took for one iteration
learning_rate = 0.001  # The speed of convergence

## Download MNIST Dataset

In [3]:
train_dataset = dsets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

## Load the Dataset

We shuffle the loading process of train_dataset to make the learning process independent of data orderness, but the order of test_loader remains to examine whether we can handle unspecified bias order of inputs.

In [4]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

## Feedforward Neural Network Model Structure

In [5]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 784 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 500 (hidden node) -> 10 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

## Instantiate the FNN

In [6]:
net = Net(input_size, hidden_size, num_classes)

## Enable GPU 

In [7]:
# net.cuda()    # You can comment out this line to disable GPU

## Choose the Loss Function and Optimizer

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

## Train the FNN Model

In [9]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):   # Load a batch of images with its (index, data, class)
        images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        labels = Variable(labels)
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(images)                             # Forward pass: compute the output class given a image
        loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        if (i+1) % 100 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/5], Step [100/600], Loss: 0.3398
Epoch [1/5], Step [200/600], Loss: 0.4684
Epoch [1/5], Step [300/600], Loss: 0.2309
Epoch [1/5], Step [400/600], Loss: 0.2741
Epoch [1/5], Step [500/600], Loss: 0.2412
Epoch [1/5], Step [600/600], Loss: 0.0925
Epoch [2/5], Step [100/600], Loss: 0.0504
Epoch [2/5], Step [200/600], Loss: 0.1099
Epoch [2/5], Step [300/600], Loss: 0.1431
Epoch [2/5], Step [400/600], Loss: 0.1107
Epoch [2/5], Step [500/600], Loss: 0.0786
Epoch [2/5], Step [600/600], Loss: 0.1334
Epoch [3/5], Step [100/600], Loss: 0.0888
Epoch [3/5], Step [200/600], Loss: 0.0623
Epoch [3/5], Step [300/600], Loss: 0.1323
Epoch [3/5], Step [400/600], Loss: 0.0411
Epoch [3/5], Step [500/600], Loss: 0.1092
Epoch [3/5], Step [600/600], Loss: 0.1109
Epoch [4/5], Step [100/600], Loss: 0.0330
Epoch [4/5], Step [200/600], Loss: 0.0405
Epoch [4/5], Step [300/600], Loss: 0.0389
Epoch [4/5], Step [400/600], Loss: 0.0482
Epoch [4/5], Step [500/600], Loss: 0.0200
Epoch [4/5], Step [600/600], Loss:

## Test the FNN Model

Similar to training the nerual network, we also need to load batches of test images and collect the outputs. The differences are that:
(1) No loss & weights calculation
(2) No wights update
(3) Has correct prediction calculation

In [10]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
    total += labels.size(0)                    # Increment the total count
    correct += (predicted == labels).sum()     # Increment the correct count
    
print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10K test images: 97 %


## Save the trained FNN Model for future use

In [11]:
# Save the model
torch.save(net.state_dict(), 'fnn_model.pkl')

### jeff